In [ ]:
!pip install langchain langchain-openai python-dotenv --quiet
print("✅ Libraries installed.")

import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool

# Configure the OpenAI API Key
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API Key set successfully from Colab Secrets.")
except (ImportError, userdata.SecretNotFoundError):
    from dotenv import load_dotenv
    load_dotenv()
    if "OPENAI_API_KEY" in os.environ:
        print("✅ OpenAI API Key set successfully from .env file.")
    else:
        print("⚠️ API Key not found. Please set it in Colab Secrets or a .env file.")

✅ Libraries installed.
✅ OpenAI API Key set successfully from Colab Secrets.


In [ ]:
EMPLOYEES = {
    "EMP001": {
        "name": "Priya Sharma",
        "department": "Engineering",
        "leave_balance": 15,
        "sick_leave": 5,
        "joined_date": "2022-01-15"
    },
    "EMP002": {
        "name": "Rahul Verma",
        "department": "Marketing",
        "leave_balance": 20,
        "sick_leave": 8,
        "joined_date": "2021-06-01"
    },
    "EMP003": {
        "name": "Anjali Patel",
        "department": "HR",
        "leave_balance": 18,
        "sick_leave": 6,
        "joined_date": "2020-03-10"
    }
}

POLICIES = {
    "leave_policy": """
    Annual Leave Policy:
    - Employees get 20 days of annual leave per year.
    - Leave must be requested at least 2 weeks in advance.
    - Maximum consecutive leave: 15 days.
    - Unused leave carries over up to 5 days.
    """,
    "work_from_home": """
    Work From Home Policy:
    - Employees can work from home up to 3 days per week.
    - Must inform manager 1 day in advance.
    - Core hours: 10 AM - 4 PM must be available.
    """,
    "benefits": """
    Employee Benefits:
    - Health insurance (employee + family).
    - Annual performance bonus.
    - Professional development budget: ₹1,50,000/year.
    - Gym membership reimbursement.
    - Festival bonuses (Diwali, etc.).
    """
}
print("✅ Mock HR data and policies defined.")

✅ Mock HR data and policies defined.


In [ ]:
@tool
def get_leave_balance(employee_id: str) -> str:
    """Gets the available leave balance for a specified employee."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']} has {emp['leave_balance']} annual leave days and {emp['sick_leave']} sick leave days remaining."
    return f"Employee with ID '{employee_id}' was not found in the database."

@tool
def get_company_policy(policy_name: str) -> str:
    """Retrieves the text of a specific company policy."""
    policy_key = policy_name.lower().replace(" ", "_")
    if policy_key in POLICIES:
        return POLICIES[policy_key]
    return f"Policy '{policy_name}' not found. Available policies are: {', '.join(POLICIES.keys())}"

@tool
def request_leave(employee_id: str, leave_days: int, start_date: str) -> str:
    """Submits a leave request for an employee."""
    if employee_id not in EMPLOYEES:
        return f"Employee with ID '{employee_id}' was not found."

    emp = EMPLOYEES[employee_id]
    if leave_days > emp['leave_balance']:
        return f"Leave request DENIED. {emp['name']} only has {emp['leave_balance']} days available, but requested {leave_days}."

    if leave_days > 15:
        return "Leave request DENIED. The maximum number of consecutive leave days allowed is 15."

    return f"Leave request for {emp['name']} for {leave_days} days starting {start_date} has been successfully submitted for manager approval."

@tool
def get_employee_info(employee_id: str) -> str:
    """Gets basic information for a specified employee, such as their name, department, and joining date."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"Employee Name: {emp['name']}\nDepartment: {emp['department']}\nJoined Date: {emp['joined_date']}"
    return f"Employee with ID '{employee_id}' was not found."

tools = [get_leave_balance, get_company_policy, request_leave, get_employee_info]
print("✅ HR tools defined and ready for the agent.")

✅ HR tools defined and ready for the agent.


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and friendly HR assistant. Your goal is to assist employees with their HR-related queries accurately and professionally. Use the available tools to find information and perform actions. If you cannot answer a question, say so politely."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ HR Agent configured and ready.")

✅ HR Agent configured and ready.


In [ ]:
def main():
    print("="*60)
    print("🤖 Welcome to the Interactive HR Assistant!")
    print("="*60)
    print("\nI can help you with:")
    print("- Leave balances (e.g., 'What is the leave balance for EMP001?')")
    print("- Company policies (e.g., 'Tell me about the work from home policy')")
    print("- Leave requests (e.g., 'Request 5 days of leave for EMP002 starting next Monday')")
    print("- Employee information (e.g., 'Who is EMP003?')")
    print("\nType 'quit' or 'exit' to end the conversation.\n")

    chat_history = []

    while True:
        user_input = input("You: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("\nHR Assistant: Thank you for using the HR Assistant. Have a great day!")
            break

        if not user_input:
            continue

        try:
            response = agent_executor.invoke({
                "input": user_input,
                "chat_history": chat_history
            })

            print(f"\nHR Assistant: {response['output']}\n")

            chat_history.append(HumanMessage(content=user_input))
            chat_history.append(AIMessage(content=response['output']))

        except Exception as e:
            print(f"\nAn error occurred: {e}\n")

if __name__ == "__main__":
    main()

🤖 Welcome to the Interactive HR Assistant!

I can help you with:
- Leave balances (e.g., 'What is the leave balance for EMP001?')
- Company policies (e.g., 'Tell me about the work from home policy')
- Leave requests (e.g., 'Request 5 days of leave for EMP002 starting next Monday')
- Employee information (e.g., 'Who is EMP003?')

Type 'quit' or 'exit' to end the conversation.

You: What is the leave balance for EMP001?


> Entering new AgentExecutor chain...

Invoking: `get_leave_balance` with `{'employee_id': 'EMP001'}`


Priya Sharma has 15 annual leave days and 5 sick leave days remaining.Priya Sharma has a leave balance of 15 annual leave days and 5 sick leave days remaining.

> Finished chain.

HR Assistant: Priya Sharma has a leave balance of 15 annual leave days and 5 sick leave days remaining.

You: Tell me about the work from home policy


> Entering new AgentExecutor chain...

Invoking: `get_company_policy` with `{'policy_name': 'work from home'}`



    Work From Home Policy:

Test